In [29]:
require 'nn'
require 'image'
require 'cutorch'
require 'cunn'
require 'optim'
require 'csvigo'

In [30]:
torch.setdefaulttensortype('torch.FloatTensor')

Prepare data:

In [9]:
function load_train_dataset(path)
    local raw_data = csvigo.load{path = path, mode = 'raw'} 
    local loaded_data = {}
    
    local data = torch.Tensor(#raw_data - 1, #raw_data[1])
    
    for i = 2, #raw_data do
        data[i - 1] = torch.Tensor(raw_data[i])
    end
    
    local labels = data[{{}, 1}]
    labels[torch.eq(labels, 0)] = 10
    local predictors = normalize(data[{{}, {2, -1}}])
    
    return {labels=labels,
            predictors=predictors,
            size=function() return (#labels)[1] end}
end

function load_test_dataset(path, train_data)
    local raw_data = csvigo.load{path = path, mode = 'raw'} 
    local loaded_data = {}
    
    local data = torch.Tensor(#raw_data - 1, #raw_data[1])
    
    for i = 2, #raw_data do
        data[i - 1] = torch.Tensor(raw_data[i])
    end
    
    mean = train_data.predictors:mean()
    std = train_data.predictors:std()
    local predictors = normalize(data, mean, std)
    
    return {predictors=predictors,
            size=function() return (#labels)[1] end}
end

function normalize(data, mean, std)
    local mean = mean or data:mean()
    local std = std or data:std()
    return (data - mean)/std
end

train_data = load_train_dataset('~/Downloads/train.csv')
test_data = load_test_dataset('~/Downloads/test.csv', train_data)

<csv>	parsing file: /Users/vitillo/Downloads/train.csv	


<csv>	parsing done	


<csv>	parsing file: /Users/vitillo/Downloads/test.csv	


<csv>	parsing done	


Setup model:

In [10]:
function setup_model(use_cuda)
    local model = nn.Sequential()
    model:add(nn.View(1, 28, 28))
    model:add(nn.SpatialConvolutionMM(1, 8, 5, 5, 1, 1, 2))
    model:add(nn.ReLU())
    model:add(nn.SpatialMaxPooling(2,2,2,2))
    
    model:add(nn.SpatialConvolutionMM(8, 16, 5, 5, 1, 1, 2))
    model:add(nn.ReLU())
    model:add(nn.SpatialMaxPooling(3,3,3,3))
    
    model:add(nn.View(16*4*4))
    model:add(nn.Linear(16*4*4, 10))
    model:add(nn.LogSoftMax())

    local criterion = nn.ClassNLLCriterion()
    if use_cuda then
        model:cuda()
        criterion:cuda()
    end
    return model, criterion
end

model, criterion = setup_model(false)

In [13]:
model = torch.load("mnist_model")  -- load pre-trained model if available

Train!

In [31]:
classes = {'1','2','3','4','5','6','7','8','9','0'}

function minibatch(train_data, shuffle, t, batchSize)
    local inputs = {}
    local targets = {}
    
    for i = t,math.min(t + batchSize - 1, train_data:size()) do
        local input = train_data.predictors[shuffle[i]]
        if useCuda then input = input:cuda() end
        local target = train_data.labels[shuffle[i]]
        table.insert(inputs, input)
        table.insert(targets, target)
    end
    
    return inputs, targets
end
function train_epoch(train)
    epoch = epoch or 1
    
    local time = sys.clock()
    local shuffle = torch.randperm(train:size())
    local parameters, gradParameters = model:getParameters()
    local confusion = optim.ConfusionMatrix(classes)
    
    local batchSize = 25
    local optimState = {
        learningRate = 1e-2,
        weightDecay = 0.001,
        momentum = 0.9,
        learningRateDecay = 1e-7
    }
    
    print("==> online epoch # " .. epoch .. ' [batchSize = ' .. batchSize .. ']')
    
    for t = 1,train:size(),batchSize do
        inputs, targets = minibatch(train, shuffle, t, batchSize)
        
        local feval = function(x)
            -- get new parameters
            if x~= parameters then
                parameters:copy(x)
            end
            
            -- reset gradients
            gradParameters:zero()
            
            -- f is the average of all criterions
            local f = 0
            
            -- evaluate function for complete mini batch
            for i = 1,#inputs do
               -- estimate f
                local output = model:forward(inputs[i])
                local err = criterion:forward(output, targets[i])
                f = f + err
                
                -- estimate df/dW
                local df_do = criterion:backward(output, targets[i])
                model:backward(inputs[i], df_do)
                                
                -- update confusion
                confusion:add(output, targets[i])
            end
            
            -- normalize gradients and f(X)
            gradParameters:div(#inputs)
            f = f/#inputs
            
            -- return f and df/dW
            return f,gradParameters
        end
        
        optim.sgd(feval, parameters, optimState)
    end
    
    -- time taken
    time = sys.clock() - time
    time = time / train:size()
    print("==> time to learn 1 sample = " .. (time*1000) .. ' ms')
    
    confusion:updateValids()
    print("==> average row correct: " .. 100*confusion.averageValid)
    confusion:zero()
    
    epoch = epoch + 1
end

In [32]:
for i=1,25 do
    train_epoch(train_data)
end

==> online epoch # 1 [batchSize = 25]	


==> time to learn 1 sample = 0.69993802479335 ms	
==> average row correct: 99.317809343338	


==> online epoch # 2 [batchSize = 25]	


==> time to learn 1 sample = 0.69234964393434 ms	
==> average row correct: 99.273147583008	


==> online epoch # 3 [batchSize = 25]	


...

==> online epoch # 24 [batchSize = 25]	


==> time to learn 1 sample = 0.69230171044668 ms	


==> average row correct: 99.616076350212	


==> online epoch # 25 [batchSize = 25]	


==> time to learn 1 sample = 0.69467242558797 ms	
==> average row correct: 99.605869054794	


Test model:

In [7]:
torch.save("mnist_model", model)

In [33]:
function max_tensor_index(tensor)
    local max_index = 1
    for i=1, tensor:size()[1] do
        if tensor[i] > tensor[max_index] then
            max_index = i
        end
    end
    
    if max_index == 10 then
        return 0
    else
        return max_index
    end
end

function test_model(test_data)
    result = {}
    
    for i = 1,test_data.predictors:size()[1] do
        local output = max_tensor_index(model:forward(test_data.predictors[i]))
        table.insert(result, output)
    end
    
    return result
end


result = test_model(test_data)

In [34]:
function write_result(path, result)
    file = io.open(path, 'w')
    file:write("ImageId,Label\n")
    
    for i = 1, #result do
        file:write(i .. ',' .. result[i] .. '\n')
    end
    file:close()
end

write_result('result.csv', result)